In [ ]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt
import pandas as pd
import tqdm
from multiprocessing import Pool

import scipy

from skimage.io import imread
from skimage.feature import greycomatrix, greycoprops
from skimage import filters
from skimage import measure

import rasterio, os, glob, warnings, time
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
plt.rcParams['font.size'] = 18
import seaborn as sns

import rioxarray as rx

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn import metrics
from sklearn.inspection import permutation_importance 

from sampling import *
from utils import *

warnings.filterwarnings("ignore")
plt.rcParams["figure.autolayout"] = True

In [ ]:
# Input image paths
basepath = 'D:\Berkeley\Research\ExplainableWildfire\datasets'
os.chdir(basepath)

img_train = sorted(glob.glob('s2*.tif'))
dem_train = sorted(glob.glob('dem*.tif'))
slope_train = sorted(glob.glob('slope*.tif'))
aoi_train = sorted(glob.glob('fire*.tif'))
lulc_train = sorted(glob.glob('lulc*.tif'))

# Weather data
rmin_train = sorted(glob.glob('rmin*.tif'))
rmax_train = sorted(glob.glob('rmax*.tif'))
tmmn_train = sorted(glob.glob('tmmn*.tif'))
tmmx_train = sorted(glob.glob('tmmx*.tif'))
vs_train = sorted(glob.glob('vs*.tif'))


In [ ]:
img_train, dem_train, slope_train, lulc_train, aoi_train = paths(basepath)

In [ ]:
paths_df = pd.DataFrame(data={'s2':img_train, 'slope': slope_train, "lulc":lulc_train, 'aoi':aoi_train, 'dem': dem_train})
print('Number of fires:', paths_df.shape[0])
paths_df.head()

In [ ]:
idx = np.random.randint(paths_df.shape[0])
idx

In [ ]:
_ = paths_df.iloc[idx]

_s2_b = rx.open_rasterio(_['s2'])[0].to_numpy()
_s2_g = rx.open_rasterio(_['s2'])[1].to_numpy()
_s2_r = rx.open_rasterio(_['s2'])[2].to_numpy()
_s2_n = rx.open_rasterio(_['s2'])[3].to_numpy()
_slope = rx.open_rasterio(_['slope']).to_numpy()[0]
_dem = rx.open_rasterio(_['dem']).to_numpy()[0]

x, y = np.gradient(_dem)
_aspect = np.arctan2(-x, y)

_lulc = rx.open_rasterio(_['lulc']).to_numpy()[0]
_aoi = rx.open_rasterio(_['aoi']).to_numpy()[0]

_ndvi = (_s2_n/65535-_s2_r/65535)/(_s2_n/65535+_s2_r/65535)

_savi = (_s2_n/65535-_s2_r/65535)/(_s2_n/65535+_s2_r/65535+0.5)*(1+0.5)

In [ ]:
# from skimage.feature import greycomatrix, greycoprops
# from multiprocessing import Pool

# def glcm_props(patch):
#     lf = []
#     props = ['dissimilarity', 'contrast', 'homogeneity', 'energy', 'correlation']

#     # left nearest neighbor
#     glcm = greycomatrix(patch, [3], [0], 256, symmetric=True, normed=True)
#     for f in props:
#         lf.append( greycoprops(glcm, f)[0,0] )

#     # upper nearest neighbor
#     glcm = greycomatrix(patch, [3], [np.pi/2], 256, symmetric=True, normed=True)
#     for f in props:
#         lf.append( greycoprops(glcm, f)[0,0] )
        
#     return lf

# def patch_gen(img, PAD=4):
#     img1 = (img * 255).astype(np.uint8)

# #     W = 512
#     W, H = img.shape[0], img.shape[1]
#     imgx = np.zeros((W+PAD*2, H+PAD*2), dtype=img1.dtype)
#     imgx[PAD:W+PAD,PAD:H+PAD] = img1
#     imgx[:PAD,  PAD:H+PAD] = img1[PAD:0:-1,:]
#     imgx[-PAD:, PAD:H+PAD] = img1[W-1:-PAD-1:-1,:]
#     imgx[:, :PAD ] = imgx[:, PAD*2:PAD:-1]
#     imgx[:, -PAD:] = imgx[:, H+PAD-1:-PAD*2-1:-1]

#     xx, yy = np.meshgrid(np.arange(0, W), np.arange(0, W))
#     xx, yy = xx.flatten() + PAD, yy.flatten() + PAD

#     for x, y in zip(xx, yy):
#         patch = imgx[y-PAD:y+PAD+1, x-PAD:x+PAD+1]
#         return patch

# def glcm_feature(img, verbose=False):
    
# #     W, NF, PAD = 512, 10, 4
    
#     W, H, NF, PAD = img.shape[0], img.shape[1], 10, 4

#     if img.sum() == 0:
#         return np.zeros((W,H,NF), dtype=np.float32)
    
#     l = []
#     with Pool(10) as pool:
#         for p in tqdm.tqdm(pool.map(glcm_props, patch_gen(img, PAD)), total=W*H, disable=not verbose):
#             l.append(p)
        
#     fimg = np.array(l, dtype=np.float32).reshape(img.shape[0], img.shape[1], -1)
#     return fimg

In [1]:
import glcm_feature

In [ ]:
_ndvi.shape

In [ ]:
glcm_feature(_ndvi)